# Sugrl Tutorial
#### This tutorial illustrates the use of Sugrl algorithm [Simple Unsupervised Graph Representation Learning](https://ojs.aaai.org/index.php/AAAI/article/view/20748), a novel unsupervised graph representation learning method that employs a novel multiplet loss to leverage the complementary nature of structural and neighbor information, thereby enhancing inter-class variation. Additionally, it incorporates an upper bound loss to maintain a finite distance between positive and anchor embeddings, effectively reducing intra-class variation. This approach sidesteps the need for data augmentation and discriminators, enabling the production of efficient, low-dimensional embeddings.
#### The tutorial is organized as folows:
#### 1. [Preprocessing Data and Loading Configuration](InfoGraph.ipynb#L48)
#### 2. [Training the model](InfoGraph.ipynb#L100)
#### 3. [Evaluating the model](InfoGraph.ipynb#L206)

## 1. Preprocessing Data and Loading Configuration 
#### First, we load the configuration from yml file and the dataset. 
#### For easy usage, we conduct experiments to search for the best parameter across three datasets and find the proper value of parameters such that the performance of implemented InfoGraph is similar to the value reported in the paper.

In [ ]:
import torch
print(torch.__version__)

In [ ]:
import torch 
from src.augment import RandomMask, RandomDropEdge, RandomDropNode, AugmentSubgraph, AugmentorList
from src.methods import SugrlMLP, SugrlGCN
from src.methods import SUGRL
from src.trainer import SimpleTrainer
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T
from src.transforms import NormalizeFeatures, GCNNorm, Edge2Adj, Compose
from src.datasets import Planetoid, Amazon, WikiCS,Coauthor
from src.utils.create_data import create_masks
from src.evaluation import LogisticRegression
import yaml
from src.utils.add_adj import add_adj_t
from sklearn.impute import SimpleImputer
import os

In [ ]:
# load the configuration file
# config = yaml.safe_load(open('./configuration/sugrl_wikics.yml', 'r', encoding='utf-8').read())
config = yaml.safe_load(open("./configuration/sugrl_amazon.yml", 'r', encoding='utf-8').read())
# config = yaml.safe_load(open("./configuration/sugrl_coauthor.yml", 'r', encoding='utf-8').read())
# config = yaml.safe_load(open("./configuration/sugrl_cora.yml", 'r', encoding='utf-8').read())
print(config)
data_name = config['dataset']
torch.manual_seed(0)
# np.random.seed(config.torch_seed)
# device = torch.device("cuda:{}".format(config.gpu_idx) if torch.cuda.is_available() and config.use_cuda else "cpu")

# -------------------- Data --------------------
pre_transforms = Compose([NormalizeFeatures(ord=1), Edge2Adj(norm=GCNNorm(add_self_loops=1))])
data_name = config['dataset']

current_folder = os.path.abspath('')
# path = os.path.join(current_folder, config.dataset.root, config.dataset.name)

if data_name=="cora":
    dataset = Planetoid(root="pyg_data", name="cora", pre_transform=pre_transforms)
if data_name=="photo": #92.9267
    dataset = Amazon(root="pyg_data", name="photo", pre_transform=pre_transforms) 
elif data_name=="coauthor": # 92.0973
    dataset = Coauthor(root="pyg_data", name='cs', transform=pre_transforms)
elif data_name=="wikics": #82.0109
    dataset = WikiCS(root="pyg_data", transform=T.NormalizeFeatures())
    dataset = add_adj_t(dataset)
    nan_mask = torch.isnan(dataset[0].x)
    imputer = SimpleImputer()
    dataset[0].x = torch.tensor(imputer.fit_transform(dataset[0].x))


# dataset = Amazon(root="pyg_data", name="photo", pre_transform=pre_transforms)
data_loader = DataLoader(dataset)
data = dataset.data

## 2. Training the Model
#### In the second step, we first initialize the parameters of Sugrl. The backbones of the encoder are MLP and GCN. 
#### You may replace the encoder with the user-defined encoder. Keep in mind that the encoder consists of class initialization, forward function, and get_embs() function.

In [ ]:
# ------------------- Method -----------------
encoder_1 = SugrlMLP(in_channels=data.x.shape[1])
encoder_2 = SugrlGCN(in_channels=data.x.shape[1])
method = SUGRL(encoder=[encoder_1,encoder_2],data = data, config=config,device="cuda:0")

#### We train the model by calling the trainer.train() function.

In [ ]:
trainer = SimpleTrainer(method=method, data_loader=data_loader, device="cuda:0")
trainer.train()

## 3. Evaluating the performance of Sugrl
#### In the last step, we evaluate the performance of Sugrl. We first get the embedding by calling method.get_embs() function and then use logistic regression to evaluate its performance. 
#### The more choice of classifiers can be found in [classifier.py](https://github.com/IDEA-ISAIL/ssl/edit/molecure/src/evaluation/classifier.py), including SVM, RandomForest, etc. 
#### Besides, other evaluation methods in an unsupervised setting could be found in [cluster.py](https://github.com/IDEA-ISAIL/ssl/edit/molecure/src/evaluation/cluster.py) or [sim_search.py](https://github.com/IDEA-ISAIL/ssl/edit/molecure/src/evaluation/sim_search.py), including K-means method or similarity search.

In [ ]:
# ------------------ Evaluator -------------------
data_pyg = dataset.data.to(method.device)
embs = method.get_embs(data_pyg.x, data_pyg.adj_t).detach()
lg = LogisticRegression(lr=0.001, weight_decay=0, max_iter=3000, n_run=50, device="cuda")
create_masks(data=data_pyg.cpu())
lg(embs=embs, dataset=data_pyg)